# 📌 id값 추가
- 이후 값 추가 시 ai함수 생성 필요

### ✏️ local DB 연결(TEST용)

In [2]:
import pymongo

conn = pymongo.MongoClient("localhost", 27017)

db_soaeng = conn.soaeng
dbcol_origin = db_soaeng.words
dbcol_words = db_soaeng.voca

# db_bigvoca = conn.bigvoca
# dbcol_voca = db_bigvoca.bigvoca_words

words = list(dbcol_word.find({}).sort("frequency", -1))
for idx, word in enumerate(words):
    word["voca_id"] = idx + 1
    dbcol_word.update_one({"_id": word["_id"]}, {"$set": word})

### ✏️ ChuAnyOne DB 연결

In [26]:
import pymongo
import dotenv
import os

# 환경변수 불러오기
dotenv.load_dotenv(dotenv.find_dotenv())
USER = os.environ["MONGODB_USER"] # MongoDB user
PASSWORD = os.environ["MONGODB_PW"] # MongoDB password
PORT = int(os.environ["MONGODB_PORT"]) # MongoDB port

# DB 연결
client = pymongo.MongoClient("mongodb://" + USER + ":" + PASSWORD + "@j7e104.p.ssafy.io", PORT)
db_bigvoca = client.bigvoca
dbcol_voca = db_bigvoca.words

# 빈도수 순으로 정렬
words = list(dbcol_voca.find({}).sort("frequency", -1))

# voca_id 추가
for idx, word in enumerate(words):
    word["voca_id"] = idx + 1
    dbcol_voca.update_one({"_id": word["_id"]}, {"$set": word})